In [ ]:
import os
str1 = ''
file_list = []

os.chdir(r"F:\河南职务侵占清洗")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
print('\n'.join(file_list))


import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'C:\\Users\\ThinkPad\\Desktop\\河南\\合并\\'

for i in range(len(file_list)):
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'GB18030')
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    else:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

import pandas as pd
import numpy as np
import re
import datetime

all_流水 = pd.DataFrame()

for i in range(len(file_list)):
    Efile =  pd.ExcelFile(file_list[i])
    sheet_list = pd.ExcelFile(file_list[i]).sheet_names
    for k in range(0,len(sheet_list)):
        if sheet_list[k] != '':
            df = Efile.parse(sheet_name = sheet_list[k],dtype = 'object')
            df['path'] = file_list[i]
            all_流水 = all_流水.append(df)
            print(str(i)+'-'+str(k),end = ' ')
        else:
            continue
all_流水.to_excel(output_path + '合并.xlsx',index = None)
print('Done!')

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import os

output_path = r'C:\Users\ThinkPad\Desktop\河南第二批'
output_path = eval(repr(output_path))+str('\\')

df = pd.read_excel(r"C:\Users\ThinkPad\Desktop\河南第二批\合并\银行交易流水.xlsx",encoding = "gbk",dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))

df['序号'] = df.index
# 24小时标记
df['计时标记'] = ''
all_流水完整 = df

def 计时12to24标记(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['交易时间'].iloc[k][11:19] < df_计时['交易时间'].iloc[(k1)][11:19]):
            df_计时['计时标记'].iloc[k] = '+12'
    return df_计时

def 计时24之后标记(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['计时标记'].iloc[(k1)]=='+12'):
            df_计时['计时标记'].iloc[k] = '+12'
    return df_计时


def index向前N位的index(df_序列, j_一方, no_N):
    for k in df_序列.index:
        if k >j_一方:
            no_序列号 = len(df_序列.loc[:k] )
            no_目标序列号 = no_序列号 - no_N
            if no_目标序列号 <= 0:
                index_目标 = df_序列.index.min()
            else:
                index_目标 = df_序列.reset_index().iloc[ no_目标序列号-1 ]['index']
            break
    return index_目标

def 一方匹配另一方(df_一方, all_另一方,tablename_索引,sheetname_层数):
    df_一方.index = df_一方['序号'].to_list()
    df_一方 = df_一方[df_一方['借贷标志']=='出'].drop_duplicates(subset = ['序号'],keep = 'last')
    df_匹配 = pd.DataFrame()
    df_未匹配 = pd.DataFrame()
    df_另一方 = pd.DataFrame(all_另一方[ (all_另一方['借贷标志']=='进')  ])
    print(str(len(df_一方))+"条，最大索引为"+ str(df_一方.index.max()))

    for j in df_一方.index:
        for N in range(100):
            x = index向前N位的index(df_另一方,j, N)
    
            df_一出一进 = df_另一方[( df_另一方['序号'] >= x ) & ( df_另一方['序号'] < j ) \
                & (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \
                ]
            if df_一方['交易金额'].loc[j]*(1-0.10) < df_一出一进['交易金额'].sum(): ############下浮10%###########
                break
            else:
                continue
        if len(df_一出一进)>0:

            
            merge_一出一进 = pd.merge(df_一方.loc[[j],:],df_一出一进,how = 'outer')
            df_匹配 = df_匹配.append(merge_一出一进)
            # 24小时标记
            df_匹配 = 计时12to24标记(df_匹配)
            df_匹配 = 计时24之后标记(df_匹配)
            # 24小时标记
            print(str(j)+"-"+str(x), end = ' ')
        elif len(df_一出一进)==0:
            df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
            print(str(j)+'失败', end = ' ')

    if not os.path.isfile(output_path + tablename_索引+ "追踪来源.xlsx"):
        df_匹配.to_excel(output_path + tablename_索引+ "追踪来源.xlsx", sheet_name= sheetname_层数,index = False)
    else:
        with pd.ExcelWriter(output_path + tablename_索引+ "追踪来源.xlsx" ,mode='a', engine='openpyxl') as writer:
            df_匹配.to_excel(writer, sheet_name= sheetname_层数,index = False)
    return df_匹配

def AB匹配(df_A,df_B,tablename_索引,sheetname_层数):
    df_A.index = df_A['序号'].to_list()
    df_A进B出 = pd.DataFrame()
    df_A进B空 = pd.DataFrame()
    df_A进 = df_A[df_A['借贷标志']=='进'].drop_duplicates(subset = ['序号'],keep = 'last')
    for k in df_A进.index:
        df_B出 = df_B[(df_B['借贷标志']=='出') \
            & ( df_B['交易对方卡号'].isin(df_A进['查询卡号']) ) \
            & (df_B['交易时间']<= str(datetime.datetime.strptime(df_A进['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds = 10) )  ) \
            & (df_B['交易时间']>= str(datetime.datetime.strptime(df_A进['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes = -1) ) ) \
            & (df_B['交易金额'] == df_A进['交易金额'].loc[k])  \
            ]
        if len(df_B出)>0:
            # 24小时标记
            df_B出['计时标记'] = df_A进['计时标记'].loc[k]
            # 24小时标记
            merge_A进B出 = pd.merge(df_A进.loc[[k],:], df_B出, how = 'outer')
            df_A进B出 = df_A进B出.append(merge_A进B出)
            print(k, end = ' ')
        elif len(df_B出)==0:
            df_B出 = df_B[(df_B['借贷标志']=='出') \
            & ( df_B['交易对方卡号'].isin(df_A进['查询卡号']) ) \
            & (df_B['交易时间']<= str(datetime.datetime.strptime(df_A进['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S')+datetime.timedelta(hours = 12) +datetime.timedelta(seconds = 10) )  ) \
            & (df_B['交易时间']>= str(datetime.datetime.strptime(df_A进['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S')+datetime.timedelta(hours = 12) + datetime.timedelta(minutes = -1) ) ) \
            & (df_B['交易金额'] == df_A进['交易金额'].loc[k])  \
            ] 
            if len(df_B出)>0:
                merge_A进B出 = pd.merge(df_A进.loc[[k],:], df_B出, how = 'outer')
                df_A进B出 = df_A进B出.append(merge_A进B出)
                print(k, end = ' ')
            elif len(df_B出)==0:
                df_A进B空 = df_A进B空.append(df_A进.loc[[k],:])
                print(str(k)+'失败', end = ' ')
#     if not os.path.isfile(output_path + tablename_索引+ "转移未调.xlsx"):
#         df_A出B空.to_excel(output_path + tablename_索引+ "转移未调.xlsx", sheet_name= sheetname_层数,index = False)
#     else:
#         with pd.ExcelWriter(output_path + tablename_索引+ "转移未调.xlsx" ,mode='a', engine='openpyxl') as writer:
#             df_A出B空.to_excel(writer, sheet_name= sheetname_层数,index = False)
#     if not os.path.isfile(output_path + tablename_索引+ "转移已调.xlsx"):
#         df_A出B进.to_excel(output_path + tablename_索引+ "转移已调.xlsx", sheet_name= sheetname_层数,index = False)
#     else:
#         with pd.ExcelWriter(output_path + tablename_索引+ "转移已调.xlsx" ,mode='a', engine='openpyxl') as writer:
#             df_A出B进.to_excel(writer, sheet_name= sheetname_层数,index = False)
    return df_A进B出

def digui多层(df_某层,tablename_索引):
    if len(df_某层)>0:
        df_某层A进B出 = AB匹配(df_某层,all_流水完整,tablename_索引,'上一层')
        if len(df_某层A进B出)>0:
            df_上一层 = 一方匹配另一方(df_某层A进B出,all_流水完整,tablename_索引,'上一层')
            digui多层(df_上一层,tablename_索引)
        
def 合并多sheet24(多sheet_filename,tablename_索引):
    all_流水 = pd.DataFrame()
    Efile =  pd.ExcelFile(多sheet_filename)
    sheet_list = pd.ExcelFile(多sheet_filename).sheet_names
    for j in range(0,len(sheet_list)):
        if sheet_list[j] != '':
            df = Efile.parse(sheet_name = sheet_list[j],dtype = 'object')
            df['交易时间'] = df.apply(lambda x: str(datetime.datetime.strptime(x['交易时间'], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours = 12) ) if x['计时标记']=='+12' else x['交易时间'],axis=1)
            all_流水 = all_流水.append(df)
            
            print(str('')+'-'+str(j),end = ' ')
        else:
            continue
    all_流水.to_excel(output_path + tablename_索引+ "追踪来源合并.xlsx",index = False)

In [2]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=10000000 )
#                  & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')
                 & (all_流水完整['交易对方卡号']!='')]
for i in df_原点线索.index:
    print(str(i)+'开始')
    t_索引 = str(i)
    digui多层(df_原点线索.loc[[i],:],t_索引)
    file_name_t = output_path + t_索引+ "追踪来源.xlsx"
    if os.path.exists(file_name_t):
        合并多sheet24(file_name_t,t_索引)

102326开始
102326 

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1条，最大索引为69890
69890-69880 69880失败 69881失败 69882失败 69883失败 69884失败 69885失败 69886失败 69887失败 69888失败 69889 1条，最大索引为66381
66381-66380 66380失败 -0 -1 102377开始
102377失败 102384开始
102384失败 102404开始
102404 1条，最大索引为103383
103383-103382 103382失败 -0 102406开始
102406 1条，最大索引为69978
69978-69977 69977 1条，最大索引为66424
66424-66423 66423失败 -0 -1 102410开始
102410失败 102411开始
102411失败 102414开始
102414失败 102446开始
102446失败 102447开始
102447失败 102472开始
102472 1条，最大索引为102236
102236-102235 102235失败 -0 102475开始
102475 1条，最大索引为70023
70023-70013 70013失败 70014失败 70015失败 70016失败 70017失败 70019失败 70020失败 70021失败 70022失败 -0 102594开始
102594 1条，最大索引为103371
103371-103369 103369 103370失败 1条，最大索引为102591
102591-102540 102540失败 102541失败 102542失败 102543失败 102544失败 102545失败 102546失败 102547失败 102548失败 102549失败 102550失败 102551失败 102552失败 102553失败 102554失败 102555失败 102556失败 102557失败 102558失败 102559失败 102560失败 102561失败 102562失败 102563失败 102564失败 102565失败 102566失败 102567失败 102568失败 102569失败 102570失败 102571失败 102572失败 102573失败 102574失败 102575

93627-93625 93625 1条，最大索引为103983
103983-103981 103981 1条，最大索引为86939
86939-86938 86938 1条，最大索引为30719
30719-30716 30716失败 30717失败 -0 -1 -2 -3 -4 -5 103991开始
103991 1条，最大索引为86970
86970-86959 86959 86960 86961失败 86962失败 86963失败 86964失败 86965失败 86966失败 86968 86969失败 3条，最大索引为103990
103986-103985 103987-103985 103990-103988 103985 103988 2条，最大索引为93627
93627-93625 86967-86958 93625 86958失败 86959 86960 86961失败 86962失败 86963失败 86964失败 86965失败 86966失败 3条，最大索引为103987
103983-103981 103986-103985 103987-103985 103981 103985 2条，最大索引为93627
86939-86938 93627-93625 86938 93625 2条，最大索引为103983
30719-30716 103983-103981 30716失败 30717失败 103981 1条，最大索引为86939
86939-86938 86938 1条，最大索引为30719
30719-30716 30716失败 30717失败 -0 -1 -2 -3 -4 -5 -6 -7 103994开始
103994 1条，最大索引为86973
86973-86963 86963失败 86964失败 86965失败 86966失败 86968 86969失败 86972 2条，最大索引为103992
103990-103988 103992-103991 103988 103991 2条，最大索引为86970
86967-86958 86970-86959 86958失败 86959 86960 86961失败 86962失败 86963失败 86964失败 86965失败 86966失败 86968 86969失败 3

In [3]:
all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=50000000 )
#                  & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')].to_excel(output_path +"单笔索引.xlsx",sheet_name = '大于等于五千万',index = None)

with pd.ExcelWriter(output_path + "单笔索引.xlsx" ,mode='a', engine='openpyxl') as writer:
    all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=30000000 )
                 & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')].to_excel(writer,sheet_name = '大于等于三千万小于五千万',index = None)
    all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=10000000 )
                 & (all_流水完整['交易金额']<30000000 )
                 & (all_流水完整['借贷标志']=='进')].to_excel(writer,sheet_name = '大于等于一千万小于三千万',index = None)

In [21]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=97890000 )
                 & (all_流水完整['交易金额']<97900000 )
                 & (all_流水完整['借贷标志']=='进')]
for i in df_原点线索.index:
    print(str(i)+'开始')
    t_索引 = str(i)
    digui多层(df_原点线索.loc[[i],:],t_索引)
    file_name_t = output_path + t_索引+ "追踪来源.xlsx"
    if os.path.exists(file_name_t):
        合并多sheet24(file_name_t,t_索引)

103953开始
103953 1条，最大索引为21089
21089-21088 21088失败 -0 

In [26]:
all_流水完整.loc[69880:69889,:'交易时间']

,银行,客户名称,客户证件号码,查询账号,查询卡号,交易对方名称,交易对方证件号码,交易对方账号,交易对方卡号,交易类型,借贷标志,币种,交易金额,交易余额,交易时间
69880,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,1985000.0,3093359.23,2014-05-27 03:00:23
69881,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,9875.0,3103234.23,2014-05-27 03:01:04
69882,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,1985000.0,5088234.23,2014-05-27 03:02:35
69883,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,494000.0,5582234.23,2014-05-27 03:11:45
69884,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,286375.0,5868609.23,2014-05-27 03:13:34
69885,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,98800.0,5967409.23,2014-05-27 03:15:04
69886,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,1424237.5,7391646.73,2014-05-27 03:16:30
69887,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,988000.0,8379646.73,2014-05-27 04:48:51
69888,,蔡芳新,,,8981211000000007274,,,,,,进,人民币,1982000.0,10361646.73,2014-05-27 04:52:13
69889,,蔡芳新,,,8981211000000007274,北京白金至尊酒业有限公司,,,8981214000000001480,,进,人民币,4000000.0,14361646.73,2014-05-28 04:05:33
